### Importing

In [235]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

air_canada = pd.read_csv('../datasets/new_data.csv')

# seperate dataset into training and predicting
air_canada_to_predict = air_canada[air_canada['choice'] != air_canada['choice']]
air_canada = air_canada[air_canada['choice'] == air_canada['choice']]

X, y = air_canada.drop(['Unnamed: 0', 'id', 'ticket_id', 'choice'], axis=1), air_canada[['choice']]

y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

/tmp/ipykernel_224944/2163498253.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })


In [236]:
X.columns

Index(['od', 'origin', 'destination', 'flight_departure_datetime',
       'purchase_datetime', 'time_diff', 'trip_type', 'branded_fare',
       'number_of_pax', 'ADVS_price', 'PREF_price', 'ADVS_price/PREF_price',
       'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory',
       'pref_inv_full', 'advs_ratio', 'pref_ratio', 'total_cap'],
      dtype='object')

### Preprocessing

In [237]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [238]:
num_pipeline = make_pipeline(
  StandardScaler()
)

cat_pipeline = make_pipeline(
  OneHotEncoder()
)
preprocessing = ColumnTransformer([
    ('cat', cat_pipeline, ['od', 'trip_type', 'branded_fare', 'total_cap']),
    ('num', num_pipeline, ['time_diff', 'number_of_pax', 'ADVS_price', 'PREF_price', 'ADVS_price/PREF_price',
       'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory', 'pref_inv_full', 'advs_ratio', 'pref_ratio'])
  ], remainder='drop')

In [239]:
X_train_prep = preprocessing.fit_transform(X_train)
X_test_prep = preprocessing.transform(X_test)

In [240]:
X_train_prep[:1]

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        , -0.4596942 ,
        -0.84339526,  0.29593778, -0.43594868,  0.62346975,  0.62176888,
         0.58849565,  0.48100825, -0.79644397,  0.36959977, -0.5518041 ,
        -0.96011108]])

### Training

In [230]:
from xgboost import XGBClassifier

bst = XGBClassifier(learning_rate=.3, objective='multi:softmax')
bst.fit(X_train_prep, y_train)

y_pred = bst.predict(X_test_prep)

print("Accuracy:", accuracy_score(y_pred, y_test))
print("F1 Score:", f1_score(y_pred, y_test, average='weighted'))

Accuracy: 0.747012987012987
F1 Score: 0.7736891519283434


### Predicting

In [213]:
X_to_predict, y_to_predict = air_canada_to_predict.drop(['id', 'ticket_id', 'choice'], axis=1), air_canada_to_predict[['id']]
X_to_predict_prep = preprocessing.transform(X_to_predict)
y_to_predict['choice'] = bst.predict(X_to_predict_prep)

/tmp/ipykernel_224944/1900845532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = bst.predict(X_to_predict_prep)


In [214]:
y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})

/tmp/ipykernel_224944/1816271922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})


In [215]:
y_final = pd.concat([y_to_predict, air_canada])[['id', 'choice']]

### Saving

In [216]:
pd.DataFrame.to_csv(y_final, '../predictions/pred7.csv', index=False)